In [1]:
# df = spark.read.format('json').load("hdfs://orion11:33001/reddit/sampled_reddit/part-00000-e97c075c-b0b4-49da-b85b-f77c115b75b6-c000.json")
df = spark.read.format('json').load("hdfs://orion11:33001/reddit/sampled_reddit/*")
print(df.count())

309199315


In [2]:
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime, desc
from pyspark.sql.types import TimestampType, DateType

botExpr = "[bB][oO][tT]"

df2 = (df
        .filter(~(df.body.like("[deleted]") | df.author.rlike(botExpr))))

In [3]:
df2.show()

+--------+----------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+----------+--------------+-------+------------+-----+-----+------------+--------+-----------------+------------+---+------------+
|archived|          author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id| link_id|mod_reports|      name| parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|        subreddit|subreddit_id|ups|user_reports|
+--------+----------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+----------+--------------+-------+------------+-----+-----+------

In [4]:
df3 = df2.select("body", "subreddit", "ups")

In [5]:
df3.show()

+--------------------+-----------------+---+
|                body|        subreddit|ups|
+--------------------+-----------------+---+
|I read the title ...|        AskReddit|  1|
|Because you're ab...|        AskReddit|  1|
|All he had was a ...|         politics|  3|
|Flushing with you...|        AskReddit|  1|
|I grew up in Texa...|             IAmA|  1|
|I did this alread...|       California|  1|
|Ctrl-F: *drr* *dr...|             pics|  2|
|Damn. I was wrong...|       reddit.com|  1|
|          Me too. :/|        AskReddit|  5|
|I don't know, but...|        AskReddit|  2|
|How did you do that?|           trance|  1|
|I can haz google+...|           google|  0|
|Months?  That's a...|             IAmA|  2|
|Is that the Reddi...|         sandiego|  1|
|I got my invite, ...|       googleplus|  1|
|This sounds bette...|       technology|  1|
|Israel handled it...|        worldnews|  2|
|Yeah, I'm ruler g...|DrawingFromWithin|  3|
|I just found out ...|           gaming|  1|
|Hope you 

In [6]:
import pyspark.sql.functions as sf
from pyspark.sql.functions import col, avg
df5 = df3.groupBy("subreddit").agg(sf.round(avg(col("ups")), 2).alias('avg_ups')).orderBy(desc("avg_ups"))

In [7]:
df5.show()

+--------------------+-------+
|           subreddit|avg_ups|
+--------------------+-------+
|          DearYishan|  72.93|
|    qkme_transcriber|  60.48|
|    WillFerrellMemes|   60.0|
|           GifBattle|  56.09|
|     AnimatedUpvotes|   52.0|
|        dawsonscreek|  51.13|
|       GifTournament|  44.78|
|                 ekp|   42.0|
|            DateRape|  40.66|
|     BasementBiotech|   39.4|
|          bestof2009|  37.82|
|              rsdFAQ|   36.0|
|          ShittyIAmA|  34.19|
|       ELIAustralian|   32.6|
|     richpeoplejokes|   32.0|
|           AWSMirror|   32.0|
|Thatactuallydidha...|   32.0|
|          Bestof2011|  31.15|
|             rudrugs|  29.64|
|               Fluck|   27.0|
+--------------------+-------+
only showing top 20 rows

